selecting and reading the dataset

mi for all features (graph)

vertical split

mi on the set with label to find tthe target (lowest mi)

spearman with target to varibales that are outside the set (graph to see elbow)